In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from bs4 import BeautifulSoup
import requests
import json
import random

In [58]:
import json
import requests
from bs4 import BeautifulSoup
urls=['https://www.ebay.com/sch/260012/i.html?_nkw=men&_from=R40&_ipg=100&_oac=1&_pgn=','https://www.ebay.com/sch/260012/i.html?_nkw=women&_from=R40&_ipg=100&_oac=1&_pgn=']
results = []
for index,url in enumerate(urls):
    for i in range(1,20):
        url=url+str(i)
        res=requests.get(url)
        soup=BeautifulSoup(res.text,'html.parser')
        div=soup.find_all('div',id='srp-river-results')
        for d in div:
            elements=d.find_all('li',class_='s-item s-item__pl-on-bottom')
            for element in elements:
                title=element.find('div',class_='s-item__title').text.strip()
                meta_info=element.find('div',class_='s-item__details-section--primary')
                details = meta_info.find_all('div', class_='s-item__detail s-item__detail--primary')
                fetch_details= [d.text.strip() for d in details]
                for detail in fetch_details:
                    if 'Shipping' in detail or "delivery" in detail:
                        delivery_price = detail
                        continue
                    elif '$' in detail:
                        price = detail
                        continue
                    elif 'Buy It Now' in detail or 'Auction' in detail:
                        delivery_type = detail
                        continue
                    elif 'from' in detail or 'located' in detail:
                        location = detail
                
                image_link=element.find('div',class_='s-item__image')
                link=image_link.find('a')['href']
                image=image_link.find('div',class_='s-item__image-wrapper image-treatment').find('img')['src']
                item_data = {
                                'title': title,
                                'price': price,
                                'delivery_type': delivery_type,
                                'delivery_price': delivery_price,
                                'location': location,
                                'product_link': link,
                                'image_link': image,
                                'sex':'M' if index==0 else "F"
                            }
                results.append(item_data)


with open('ebay_clothing_data.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# Setup headless browser options
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


with open("products.json", "r", encoding="utf-8") as f:
    product_dicts = json.load(f)

# Extract only the product_link from each dict
urls = [item["product_link"] for item in product_dicts if "product_link" in item]

# Initialize the driver once (optional: you can reinitialize per URL if needed)
driver = webdriver.Chrome(options=options)

# Define CSS selectors
selectors = [
    'div.ux-section-module.section-module-',
    'div.fdbk-detail-list'
]
combined_selector = ', '.join(selectors)

# Store all results
all_results = []

for index,url in enumerate(urls):
    print(f"Processing link number: {index}")
    driver.get(url)

    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, combined_selector))
        )
    except TimeoutException:
        print(f"Timeout: Skipping {url} due to no expected elements.")
        continue

    results = []

    all_elements = driver.find_elements(By.CSS_SELECTOR, combined_selector)
    for index, element in enumerate(all_elements):
        if index == 0:
            elements = [e.text.strip() for e in element.find_elements(By.CSS_SELECTOR, 'div.ux-section-icon-with-details')]
            event_breath1 = elements[0] if len(elements) > 0 else None
            event_breath2 = elements[1] if len(elements) > 1 else None
        else:
            lis = element.find_elements(By.CSS_SELECTOR, 'li.fdbk-container')
            if lis:
                for li in lis:
                    try:
                        date = li.find_element(By.CSS_SELECTOR, 'div.fdbk-container__details__info__divide__list-bullet').text.strip()
                        comment = li.find_element(By.CSS_SELECTOR, 'div.fdbk-container__details__comment').text.strip()
                        if date or comment:
                            results.append((date, comment))
                    except NoSuchElementException:
                        continue

    # Attach URL source to result if needed
    all_results.append({
        'index':index,
        'url': url,
        'feedback': results
    })

driver.quit()

# Print all feedback from all pages
for entry in all_results:
    print(f"\nFeedback from {entry['url']}:")
    for feedback in entry['feedback']:
        print(feedback)

In [4]:
with open('ebay_clothing_data.json','r',encoding='utf-8') as f:
    men_products=json.load(f)


with open('ebay_clothing_data.json','r',encoding='utf-8') as f:
    women_products=json.load(f)

In [ ]:
with open('ebay_clothing_data.json','r',encoding='utf-8') as f:
  women_products=json.load(f)

In [ ]:
women_products=[item for item in women_products if item['sex']=="F"]

In [ ]:
women_products=random.sample(women_products,500)

In [ ]:
len(women_products)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# Setup headless browser options
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')





women_results=[]
# Initialize the driver once (optional: you can reinitialize per URL if needed)
driver = webdriver.Chrome(options=options)

# Define CSS selectors
selectors = [
    'div.ux-section-module.section-module-',
    'div.fdbk-detail-list'
]
combined_selector = ', '.join(selectors)

# Store all results
all_results = []

for index,url in enumerate(women_products):
    print(f"Processing link number: {index}")
    driver.get(url["product_link"])

    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, combined_selector))
        )
    except TimeoutException:
        print(f"Timeout: Skipping {url} due to no expected elements.")
        continue

    results = []

    all_elements = driver.find_elements(By.CSS_SELECTOR, combined_selector)
    for index, element in enumerate(all_elements):
        if index == 0:
            elements = [e.text.strip() for e in element.find_elements(By.CSS_SELECTOR, 'div.ux-section-icon-with-details')]
            event_breath1 = elements[0] if len(elements) > 0 else None
            event_breath2 = elements[1] if len(elements) > 1 else None
        else:
            lis = element.find_elements(By.CSS_SELECTOR, 'li.fdbk-container')
            if lis:
                for li in lis:
                    try:
                        date = li.find_element(By.CSS_SELECTOR, 'div.fdbk-container__details__info__divide__list-bullet').text.strip()
                        comment = li.find_element(By.CSS_SELECTOR, 'div.fdbk-container__details__comment').text.strip()
                        if date or comment:
                            results.append((date, comment))
                    except NoSuchElementException:
                        continue

    # Attach URL source to result if needed
    url['event_breath1']=event_breath1
    url['event_breath2']=event_breath2
    url['feedback']=results
    url['index']=index
    women_results.append(url)
    

driver.quit()
with open('women_results.json','w',encoding='utf-8') as f:
  json.dump(women_results,f,indent=4)

Processing link number: 0
Processing link number: 1
Processing link number: 2
Processing link number: 3
Processing link number: 4
Processing link number: 5
Processing link number: 6
Processing link number: 7
Processing link number: 8
Processing link number: 9
Processing link number: 10
Processing link number: 11
Processing link number: 12
Processing link number: 13
Processing link number: 14
Processing link number: 15
Processing link number: 16
Processing link number: 17
Processing link number: 18
Processing link number: 19
Processing link number: 20
Processing link number: 21
Processing link number: 22
Processing link number: 23
Processing link number: 24
Processing link number: 25
Processing link number: 26
Processing link number: 27
Processing link number: 28
Processing link number: 29
Processing link number: 30
Processing link number: 31
Processing link number: 32
Processing link number: 33
Processing link number: 34
Processing link number: 35
Processing link number: 36
Processing 

In [ ]:
with open('ebay_clothing_data.json','r',encoding='utf-8') as f:
  men_products=json.load(f)

In [ ]:
men_products=[item for item in men_products if item['sex']=="M"]

In [ ]:
men_products=random.sample(men_products,500)

In [ ]:
len(men_products)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# Setup headless browser options
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')



men_results=[]
# Initialize the driver once (optional: you can reinitialize per URL if needed)
driver = webdriver.Chrome(options=options)

# Define CSS selectors
selectors = [
    'div.ux-section-module.section-module-',
    'div.fdbk-detail-list'
]
combined_selector = ', '.join(selectors)

# Store all results
all_results = []

for index,url in enumerate(men_products):
    print(f"Processing link number: {index}")
    driver.get(url["product_link"])

    try:
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, combined_selector))
        )
    except TimeoutException:
        print(f"Timeout: Skipping {url} due to no expected elements.")
        continue

    results = []

    all_elements = driver.find_elements(By.CSS_SELECTOR, combined_selector)
    for index, element in enumerate(all_elements):
        if index == 0:
            elements = [e.text.strip() for e in element.find_elements(By.CSS_SELECTOR, 'div.ux-section-icon-with-details')]
            event_breath1 = elements[0] if len(elements) > 0 else None
            event_breath2 = elements[1] if len(elements) > 1 else None
        else:
            lis = element.find_elements(By.CSS_SELECTOR, 'li.fdbk-container')
            if lis:
                for li in lis:
                    try:
                        date = li.find_element(By.CSS_SELECTOR, 'div.fdbk-container__details__info__divide__list-bullet').text.strip()
                        comment = li.find_element(By.CSS_SELECTOR, 'div.fdbk-container__details__comment').text.strip()
                        if date or comment:
                            results.append((date, comment))
                    except NoSuchElementException:
                        continue

    # Attach URL source to result if needed
    url['event_breath1']=event_breath1
    url['event_breath2']=event_breath2
    url['feedback']=results
    men_results.append(url)
    url['index']=index
    

driver.quit()

with open('men_results.json','w',encoding='utf-8') as f:
  json.dump(men_results,f,indent=4)

In [ ]:
final_data=women_products+men_products

In [ ]:
with open('market_product.json','w',encoding='utf-8') as f:
    json.dump(final_data,f,indent=4)

In [2]:
############################## scrape images ###########################################################

In [8]:
import requests
import json
import os

In [5]:
with open(r"D:\BDIA\semestre 4\Analyse du web\final project\data\processed_data\market_data_processed_v1.json",'r',encoding='utf-8') as f:
    data=json.load(f)

In [9]:
for index,item in enumerate(data):
    response=requests.get(item['image_link'])
    filename = os.path.join(
        r'D:\BDIA\semestre 4\Analyse du web\final project\data\collected_data\images',
        f"{index}_{item['image_link'].split('/')[-1]}"
    )
    with open(filename, "wb") as f:
        f.write(response.content)